In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import uniform, randint


try:
    from xgboost import XGBClassifier
except ImportError:
    XGBClassifier = None

try:
    from lightgbm import LGBMClassifier
except ImportError:
    LGBMClassifier = None


In [ ]:
DATA_DIR = ""

adm_path = os.path.join(DATA_DIR, "ADMISSIONS.csv.gz")
icu_path = os.path.join(DATA_DIR, "ICUSTAYS.csv.gz")
patients_path = os.path.join(DATA_DIR, "PATIENTS.csv.gz")

adm = pd.read_csv(adm_path, compression="gzip")
icu = pd.read_csv(icu_path, compression="gzip")
patients = pd.read_csv(patients_path, compression="gzip")

adm["ADMITTIME"] = pd.to_datetime(adm["ADMITTIME"], errors="coerce")
adm["DISCHTIME"] = pd.to_datetime(adm["DISCHTIME"], errors="coerce")
icu["INTIME"] = pd.to_datetime(icu["INTIME"], errors="coerce")
icu["OUTTIME"] = pd.to_datetime(icu["OUTTIME"], errors="coerce")

adm["MORTALITY"] = adm["HOSPITAL_EXPIRE_FLAG"].astype(int)

patients["DOB"] = pd.to_datetime(patients["DOB"], errors="coerce")
patients_age = patients[["SUBJECT_ID", "DOB", "GENDER"]].copy()

In [ ]:
cohort = icu.merge(
    adm[["SUBJECT_ID","HADM_ID","MORTALITY","ADMITTIME"]],
    on=["SUBJECT_ID","HADM_ID"],
    how="left"
)

cohort = cohort.sort_values(["SUBJECT_ID","HADM_ID","INTIME"])

cohort_first = cohort.groupby(["SUBJECT_ID","HADM_ID"]).first().reset_index()
cohort_first = cohort_first.merge(patients_age, on="SUBJECT_ID", how="left")

cohort_first["DOB"] = pd.to_datetime(cohort_first["DOB"], errors="coerce")
min_valid_dob = pd.Timestamp("1920-01-01")  
valid_dob_mask = (
    cohort_first["DOB"].notna() & 
    (cohort_first["DOB"] >= min_valid_dob) & 
    (cohort_first["DOB"] <= cohort_first["ADMITTIME"]) &
    (cohort_first["ADMITTIME"] >= pd.Timestamp("2000-01-01"))
)

cohort_first["AGE"] = np.nan

if valid_dob_mask.any():
    valid_subset = cohort_first.loc[valid_dob_mask].copy()
    year_diff = valid_subset["ADMITTIME"].dt.year - valid_subset["DOB"].dt.year
    month_day_adjust = (
        (valid_subset["ADMITTIME"].dt.month * 100 + valid_subset["ADMITTIME"].dt.day) < 
        (valid_subset["DOB"].dt.month * 100 + valid_subset["DOB"].dt.day)
    ).astype(int)
    
    age_calc = (year_diff - month_day_adjust).clip(lower=0, upper=90)
    cohort_first.loc[valid_dob_mask, "AGE"] = age_calc.values

if cohort_first["AGE"].isna().any():
    median_age = cohort_first["AGE"].median()
    if pd.isna(median_age):
        median_age = 65  
    cohort_first["AGE"] = cohort_first["AGE"].fillna(median_age)

cohort_first["AGE"] = cohort_first["AGE"].clip(lower=0, upper=90)
cohort_key = cohort_first[["SUBJECT_ID","HADM_ID","ICUSTAY_ID","INTIME"]]


In [ ]:
data_base = cohort_first.merge(
    adm[[
        "SUBJECT_ID","HADM_ID",
        "ADMISSION_TYPE","ADMISSION_LOCATION",
        "DISCHARGE_LOCATION",
        "INSURANCE","MARITAL_STATUS","ETHNICITY"
    ]],
    on=["SUBJECT_ID","HADM_ID"],
    how="left"
)

data_base = data_base.merge(
    cohort_first[["SUBJECT_ID","HADM_ID","AGE","GENDER"]],
    on=["SUBJECT_ID","HADM_ID"],
    how="left"
)


In [ ]:
lab_path = os.path.join(DATA_DIR, "LABEVENTS.csv.gz")
d_labitems = pd.read_csv(os.path.join(DATA_DIR, "D_LABITEMS.csv.gz"), compression="gzip")

critical_labs = {
    "CREATININE", "LACTATE", "BILIRUBIN", "PLATELET", 
    "WBC", "HEMATOCRIT", "HEMOGLOBIN", "INR", "PTT",
    "BUN", "SODIUM", "POTASSIUM", "GLUCOSE", "PH"
}

d_labitems["LABEL_UP"] = d_labitems["LABEL"].str.upper()
lab_items = d_labitems[d_labitems["LABEL_UP"].isin(critical_labs)]
lab_itemids = set(lab_items["ITEMID"])
print("Lab ITEMIDs found:", len(lab_itemids))

lab_kept = []
lab_total_kept = 0
lab_chunk_idx = 0

for chunk in pd.read_csv(lab_path, usecols=["HADM_ID", "ITEMID", "CHARTTIME", "VALUENUM"], chunksize=300_000):
    lab_chunk_idx += 1
    if lab_chunk_idx > 200:  
        break
    
    sub = chunk[
        chunk["HADM_ID"].isin(cohort_key["HADM_ID"]) &
        chunk["ITEMID"].isin(lab_itemids) &
        chunk["VALUENUM"].notna()
    ].copy()
    
    if sub.empty:
        continue
    
    sub["CHARTTIME"] = pd.to_datetime(sub["CHARTTIME"], errors="coerce")
    sub = sub.merge(cohort_key[["HADM_ID", "ICUSTAY_ID", "INTIME"]], on="HADM_ID", how="left")
    sub["HOUR"] = (sub["CHARTTIME"] - sub["INTIME"]).dt.total_seconds()/3600
    sub = sub[(sub["HOUR"] >= 0) & (sub["HOUR"] <= 24)]
    
    if not sub.empty:
        lab_kept.append(sub[["ICUSTAY_ID", "ITEMID", "VALUENUM"]])
        lab_total_kept += len(sub)
    
    if lab_chunk_idx % 50 == 0:
        print(f"Lab chunk {lab_chunk_idx}: kept {len(sub)} rows (total {lab_total_kept})")

lab_events = pd.concat(lab_kept, ignore_index=True) if lab_kept else pd.DataFrame(
    columns=["ICUSTAY_ID", "ITEMID", "VALUENUM"]
)

print("lab_events shape:", lab_events.shape)

if not lab_events.empty:
    lab_agg = lab_events.groupby(["ICUSTAY_ID", "ITEMID"])["VALUENUM"].agg(["mean", "min", "max", "std"]).reset_index()
    
    lab_itemid_to_label = dict(zip(lab_items["ITEMID"], lab_items["LABEL_UP"]))
    
    lab_pivot = lab_agg.pivot_table(
        index="ICUSTAY_ID",
        columns="ITEMID",
        values=["mean", "min", "max", "std"]
    )
    
    lab_pivot.columns = [
        f"LAB_{lab_itemid_to_label.get(item, 'UNK').replace(' ', '_').upper()}_{stat.upper()}"
        for stat, item in lab_pivot.columns
    ]
    
    lab_wide = lab_pivot.reset_index()
    print("lab_wide shape:", lab_wide.shape)
else:
    lab_wide = pd.DataFrame(columns=["ICUSTAY_ID"])

Lab ITEMIDs found: 19
Lab chunk 50: kept 9375 rows (total 448811)
lab_events shape: (948646, 3)
lab_wide shape: (54198, 70)


In [ ]:
icu_time_map = cohort_first[["ICUSTAY_ID","INTIME"]].drop_duplicates()
icu_time_map = icu_time_map.set_index("ICUSTAY_ID")["INTIME"]
icu_ids = set(icu_time_map.index)
print("Number of ICU stays:", len(icu_ids))

d_items = pd.read_csv(os.path.join(DATA_DIR, "D_ITEMS.csv.gz"), compression="gzip")
d_items["LABEL_UP"] = d_items["LABEL"].str.upper()

target_vitals = {
    "HEART RATE",
    "SYSTOLIC BLOOD PRESSURE",
    "DIASTOLIC BLOOD PRESSURE",
    "RESPIRATORY RATE",
    "TEMPERATURE",
    "SPO2",
    "O2 SATURATION",
    "GCS"
}

vital_items = d_items[d_items["LABEL_UP"].isin(target_vitals)]
vital_itemids = set(vital_items["ITEMID"])
print("Vital ITEMIDs:", vital_itemids)

chart_path = os.path.join(DATA_DIR, "CHARTEVENTS.csv.gz")
usecols = ["ICUSTAY_ID","ITEMID","CHARTTIME","VALUENUM"]

kept = []
total_kept = 0
chunk_idx = 0

MAX_CHUNKS = 200
MAX_ROWS_KEPT = 2_000_000

for chunk in pd.read_csv(chart_path, usecols=usecols, chunksize=300_000):
    chunk_idx += 1
    if chunk_idx > MAX_CHUNKS:
        print(f"Reached MAX_CHUNKS={MAX_CHUNKS}, stopping early.")
        break

    sub = chunk[
        chunk["ICUSTAY_ID"].isin(icu_ids) &
        chunk["ITEMID"].isin(vital_itemids)
    ].copy()

    if sub.empty:
        if chunk_idx % 25 == 0:
            print(f"chunk {chunk_idx}: kept 0 rows (total {total_kept})")
        continue

    sub["CHARTTIME"] = pd.to_datetime(sub["CHARTTIME"], errors="coerce")
    sub["INTIME"] = sub["ICUSTAY_ID"].map(icu_time_map)
    sub["HOUR"] = (sub["CHARTTIME"] - sub["INTIME"]).dt.total_seconds()/3600

    sub = sub[(sub["HOUR"] >= 0) & (sub["HOUR"] <= 24)]

    if not sub.empty:
        kept.append(sub[["ICUSTAY_ID","ITEMID","VALUENUM"]])
        total_kept += len(sub)

    if chunk_idx % 25 == 0:
        print(f"chunk {chunk_idx}: kept {len(sub)} rows (total {total_kept})")

    if total_kept >= MAX_ROWS_KEPT:
        print(f"Reached MAX_ROWS_KEPT={MAX_ROWS_KEPT}, stopping early.")
        break


chart_events = pd.concat(kept, ignore_index=True) if kept else pd.DataFrame(
    columns=["ICUSTAY_ID","ITEMID","VALUENUM"]
)

print("chart_events shape:", chart_events.shape)
print(chart_events.head())

chart_agg = chart_events.groupby(["ICUSTAY_ID","ITEMID"])["VALUENUM"] \
                        .agg(["mean","min","max","std"]) \
                        .reset_index()

itemid_to_label = dict(zip(vital_items["ITEMID"], vital_items["LABEL_UP"]))

pivot = chart_agg.pivot_table(
    index="ICUSTAY_ID",
    columns="ITEMID",
    values=["mean","min","max","std"]
)

pivot.columns = [
    f"VITAL_{itemid_to_label.get(item,'UNK').replace(' ','_').upper()}_{stat.upper()}"
    for stat, item in pivot.columns
]

chart_wide = pivot.reset_index()
print("chart_wide:", chart_wide.shape)
print(chart_wide.head())


Number of ICU stays: 57786
Vital ITEMIDs: {646, 618, 220045, 220210, 211}
chunk 25: kept 13101 rows (total 279012)
chunk 50: kept 6944 rows (total 563884)
chunk 75: kept 12965 rows (total 877218)
chunk 100: kept 9882 rows (total 1196853)
chunk 125: kept 4435 rows (total 1432415)
chunk 150: kept 4615 rows (total 1541671)
chunk 175: kept 3717 rows (total 1643698)
chunk 200: kept 3476 rows (total 1758717)
Reached MAX_CHUNKS=200, stopping early.
chart_events shape: (1758717, 3)
   ICUSTAY_ID  ITEMID  VALUENUM
0    241249.0  220045      86.0
1    241249.0  220210      21.0
2    241249.0  220045      85.0
3    241249.0  220210      19.0
4    241249.0  220045      87.0
chart_wide: (27104, 21)
   ICUSTAY_ID  VITAL_HEART_RATE_MAX  VITAL_RESPIRATORY_RATE_MAX  \
0    200001.0                   NaN                         NaN   
1    200010.0                   NaN                         NaN   
2    200011.0                   NaN                         NaN   
3    200016.0                   NaN  

In [51]:
output = pd.read_csv(os.path.join(DATA_DIR, "OUTPUTEVENTS.csv.gz"), compression="gzip",
                     usecols=["SUBJECT_ID","HADM_ID","ICUSTAY_ID","CHARTTIME","VALUE"])

output["CHARTTIME"] = pd.to_datetime(output["CHARTTIME"], errors="coerce")
output = output[output["HADM_ID"].isin(cohort_key["HADM_ID"])]

output = output.merge(cohort_key, on=["SUBJECT_ID","HADM_ID","ICUSTAY_ID"], how="inner")
output["HOUR"] = (output["CHARTTIME"] - output["INTIME"]).dt.total_seconds()/3600
output = output[(output["HOUR"]>=0) & (output["HOUR"]<=24)]

urine_agg = output.groupby("ICUSTAY_ID")["VALUE"].agg(["sum","mean"]).reset_index()
urine_agg.columns = ["ICUSTAY_ID","URINE_SUM_24H","URINE_MEAN_24H"]


In [52]:
input_cv = pd.read_csv(os.path.join(DATA_DIR,"INPUTEVENTS_CV.csv.gz"), compression="gzip",
                       usecols=["SUBJECT_ID","HADM_ID","ICUSTAY_ID","ITEMID","CHARTTIME","AMOUNT"])
input_mv = pd.read_csv(os.path.join(DATA_DIR,"INPUTEVENTS_MV.csv.gz"), compression="gzip",
                       usecols=["SUBJECT_ID","HADM_ID","ICUSTAY_ID","ITEMID","STARTTIME","AMOUNT"])

input_cv["CHARTTIME"] = pd.to_datetime(input_cv["CHARTTIME"], errors="coerce")
input_mv["STARTTIME"] = pd.to_datetime(input_mv["STARTTIME"], errors="coerce")

input_cv = input_cv[input_cv["HADM_ID"].isin(cohort_key["HADM_ID"])]
input_mv = input_mv[input_mv["HADM_ID"].isin(cohort_key["HADM_ID"])]

input_cv = input_cv.merge(cohort_key, on=["SUBJECT_ID","HADM_ID","ICUSTAY_ID"])
input_mv = input_mv.merge(cohort_key, on=["SUBJECT_ID","HADM_ID","ICUSTAY_ID"])

input_cv["HOUR"] = (input_cv["CHARTTIME"] - input_cv["INTIME"]).dt.total_seconds()/3600
input_mv["HOUR"] = (input_mv["STARTTIME"] - input_mv["INTIME"]).dt.total_seconds()/3600

input_cv = input_cv[(input_cv["HOUR"]>=0)&(input_cv["HOUR"]<=24)]
input_mv = input_mv[(input_mv["HOUR"]>=0)&(input_mv["HOUR"]<=24)]

fluids = pd.concat([
    input_cv[["ICUSTAY_ID","AMOUNT"]],
    input_mv[["ICUSTAY_ID","AMOUNT"]]
])

fluids_agg = fluids.groupby("ICUSTAY_ID")["AMOUNT"].agg(["sum","mean"]).reset_index()
fluids_agg.columns = ["ICUSTAY_ID","FLUID_SUM_24H","FLUID_MEAN_24H"]


In [53]:
diag = pd.read_csv(os.path.join(DATA_DIR,"DIAGNOSES_ICD.csv.gz"), compression="gzip")

def root(code):
    if pd.isna(code): return None
    return str(code).replace(".","")[:3]

diag["ICD3"] = diag["ICD9_CODE"].apply(root)

prefixes = {
    "CMI_MI": {"410","412"},
    "CMI_CHF": {"428"},
    "CMI_COPD": {"490","491","492","494","496"},
    "CMI_DIAB": {"250"},
    "CMI_RENAL": {"585"},
    "CMI_LIVER": {"571"},
}

def map_flags(icd):
    out = {}
    for name, pref in prefixes.items():
        out[name] = int(icd in pref)
    return pd.Series(out)

diag_flags = diag["ICD3"].apply(map_flags)
diag_with_flags = pd.concat([diag["HADM_ID"], diag_flags], axis=1)
comorb = diag_with_flags.groupby("HADM_ID").max().reset_index()


In [ ]:
data_full = data_base.copy()
data_full = data_full.merge(chart_wide, on="ICUSTAY_ID", how="left")
data_full = data_full.merge(lab_wide, on="ICUSTAY_ID", how="left")  # Add lab data
data_full = data_full.merge(urine_agg, on="ICUSTAY_ID", how="left")
data_full = data_full.merge(fluids_agg, on="ICUSTAY_ID", how="left")
data_full = data_full.merge(comorb, on="HADM_ID",  how="left")

print("After merges:", data_full.shape)
data_full = data_full.loc[:, ~data_full.columns.duplicated(keep='first')]
print("After removing duplicate columns:", data_full.shape)

if "URINE_SUM_24H" in data_full.columns and "FLUID_SUM_24H" in data_full.columns:
    data_full["URINE_FLUID_RATIO"] = data_full["URINE_SUM_24H"] / (data_full["FLUID_SUM_24H"] + 1)

vital_cols = [c for c in data_full.columns if "VITAL_" in str(c)]
for col in vital_cols:
    try:
        col_idx = data_full.columns.get_loc(col)
        if isinstance(col_idx, slice):
            continue
        
        col_data = data_full.iloc[:, col_idx]
        if not pd.api.types.is_numeric_dtype(col_data):
            continue
        if "HEART_RATE" in str(col) and "MEAN" in str(col):
            new_col_name = f"{col}_TACHYCARDIA"
            if new_col_name not in data_full.columns:
                data_full[new_col_name] = (col_data > 100).astype(int)
        elif "RESPIRATORY_RATE" in str(col) and "MEAN" in str(col):
            new_col_name = f"{col}_TACHYPNEA"
            if new_col_name not in data_full.columns:
                data_full[new_col_name] = (col_data > 20).astype(int)
        elif "SPO2" in str(col) and "MIN" in str(col):
            new_col_name = f"{col}_HYPOXIA"
            if new_col_name not in data_full.columns:
                data_full[new_col_name] = (col_data < 90).astype(int)
    except (KeyError, ValueError, TypeError, IndexError) as e:
        continue

if "URINE_SUM_24H" in data_full.columns:
    data_full["HAS_URINE_DATA"] = data_full["URINE_SUM_24H"].notna().astype(int)
if "FLUID_SUM_24H" in data_full.columns:
    data_full["HAS_FLUID_DATA"] = data_full["FLUID_SUM_24H"].notna().astype(int)

print("After feature engineering:", data_full.shape)

data_full.columns = [str(c) for c in data_full.columns]

datetime_cols = data_full.select_dtypes(
    include=["datetime64[ns]", "datetime64[ns, UTC]"]
).columns.tolist()

print("Dropping datetime columns:", datetime_cols)
data_full = data_full.drop(columns=datetime_cols)

data_full = data_full.loc[:, ~data_full.columns.duplicated()]

print("After cleaning:", data_full.shape)
data_full.head()


After merges: (57786, 124)
After removing duplicate columns: (57786, 87)
After feature engineering: (57786, 93)
Dropping datetime columns: ['INTIME', 'OUTTIME', 'ADMITTIME', 'DOB']
After cleaning: (57786, 89)


,SUBJECT_ID,HADM_ID,ROW_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,LOS,...,CMI_COPD,CMI_DIAB,CMI_RENAL,CMI_LIVER,URINE_FLUID_RATIO,VITAL_HEART_RATE_MEAN_TACHYCARDIA,VITAL_RESPIRATORY_RATE_MEAN_TACHYPNEA,VITAL_SPO2_MIN_HYPOXIA,HAS_URINE_DATA,HAS_FLUID_DATA
0,2,163353,1,243653,carevue,NICU,NICU,56,56,0.0918,...,0,0,0,0,NaN,1,0,0,0,0
1,3,145834,2,211552,carevue,MICU,MICU,12,12,6.0646,...,0,0,0,0,0.031321,1,0,1,1,1
2,4,185777,3,294638,carevue,MICU,MICU,52,52,1.6785,...,0,0,0,1,0.679090,0,1,0,1,1
3,5,178980,4,214757,carevue,NICU,NICU,56,56,0.0844,...,0,0,0,0,NaN,0,0,0,0,0
4,6,107064,5,228232,carevue,SICU,SICU,33,33,3.6729,...,0,0,0,0,0.157330,0,0,1,1,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

data_full.columns = [str(c) for c in data_full.columns]

datetime_cols = data_full.select_dtypes(
    include=["datetime64[ns]", "datetime64[ns, UTC]"]
).columns.tolist()
print("Datetime columns (will be dropped from features):", datetime_cols)
target_col = "MORTALITY"

id_cols = ["SUBJECT_ID", "HADM_ID", "ICUSTAY_ID"]
leakage_cols = ["DISCHARGE_LOCATION"]   

drop_cols = [c for c in id_cols + leakage_cols + datetime_cols if c in data_full.columns]
print("Dropping from features:", drop_cols)

feature_cols = [c for c in data_full.columns if c not in drop_cols + [target_col]]
print("Number of candidate feature columns:", len(feature_cols))

X_raw = data_full[feature_cols].copy()
y = data_full[target_col].astype(int)

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, random_state=42, stratify=y
)

print("X_train_raw:", X_train_raw.shape)
print("X_test_raw:", X_test_raw.shape)
print("y_train mean:", y_train.mean())
print("y_test mean:", y_test.mean())

cat_cols = X_train_raw.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = [c for c in X_train_raw.columns if c not in cat_cols]

print("Categorical columns:", cat_cols)
print("Numeric columns count:", len(num_cols))

X_train = pd.get_dummies(X_train_raw, columns=cat_cols, drop_first=True)
X_test  = pd.get_dummies(X_test_raw,  columns=cat_cols, drop_first=True)

X_train, X_test = X_train.align(X_test, join="left", axis=1)
X_test = X_test.fillna(0)

for c in num_cols:
    if c in X_train.columns:
        med = X_train[c].median()
        X_train[c] = X_train[c].fillna(med)
        X_test[c]  = X_test[c].fillna(med)

print("After encoding + imputation:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

feature_names = X_train.columns.tolist()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


Datetime columns (will be dropped from features): []
Dropping from features: ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DISCHARGE_LOCATION']
Number of candidate feature columns: 84
X_train_raw: (46228, 84)
X_test_raw: (11558, 84)
y_train mean: 0.1005883879899628
y_test mean: 0.10062294514621907
Categorical columns: ['DBSOURCE', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'GENDER_x', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'INSURANCE', 'MARITAL_STATUS', 'ETHNICITY', 'GENDER_y']
Numeric columns count: 74
After encoding + imputation:
X_train: (46228, 149)
X_test: (11558, 149)


In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

base_clf = LogisticRegression(
    max_iter=4000,
    penalty="l2",
    solver="liblinear"
)
base_clf.fit(X_train_scaled, y_train)

base_pred = base_clf.predict_proba(X_test_scaled)[:, 1]
base_auc = roc_auc_score(y_test, base_pred)
print("Baseline AUC (all features):", base_auc)


Baseline AUC (all features): 0.4977689518532167


In [ ]:
X_train_df = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_df  = pd.DataFrame(X_test_scaled,  columns=feature_names)

X_train_df = X_train_df.loc[:, ~X_train_df.columns.duplicated()]
X_test_df  = X_test_df.loc[:, X_train_df.columns] 

feature_names = X_train_df.columns.tolist()
nunique_series = X_train_df.nunique(dropna=False)
const_cols = nunique_series[nunique_series <= 1].index.tolist()

print("Number of constant columns:", len(const_cols))

X_train_fs = X_train_df.drop(columns=const_cols)
X_test_fs  = X_test_df.drop(columns=const_cols, errors="ignore")

print("After dropping constant cols:")
print("X_train_fs:", X_train_fs.shape)
print("X_test_fs:", X_test_fs.shape)

feature_names = X_train_fs.columns.tolist()


Number of constant columns: 3
After dropping constant cols:
X_train_fs: (46228, 146)
X_test_fs: (11558, 146)


In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

max_features = X_train_fs.shape[1]
candidate_ks = [10, 20, 40, 60, 80, max_features]  # adjust if you want

print("Number of features available:", max_features)
print("Candidate K values:", candidate_ks)

results = []

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for k in candidate_ks:
    k_eff = min(k, max_features)

    mi_pipe = Pipeline([
        ("mi", SelectKBest(mutual_info_classif, k=k_eff)),
        ("clf", LogisticRegression(
            max_iter=4000,
            penalty="l2",
            solver="liblinear"
        ))
    ])

    cv_scores = cross_val_score(
        mi_pipe,
        X_train_fs,
        y_train,
        scoring="roc_auc",
        cv=cv,
        n_jobs=-1
    )

    mean_auc = cv_scores.mean()
    std_auc = cv_scores.std()
    results.append((k_eff, mean_auc, std_auc))

    print(f"k={k_eff:3d} | CV AUC={mean_auc:.4f} ± {std_auc:.4f}")

best_k, best_auc, best_std = max(results, key=lambda x: x[1])
print("\nBest K from CV:", best_k)
print("Best CV AUC   :", best_auc, "±", best_std)


Number of features available: 146
Candidate K values: [10, 20, 40, 60, 80, 146]
k= 10 | CV AUC=0.7547 ± 0.0222
k= 20 | CV AUC=0.7856 ± 0.0115
k= 40 | CV AUC=0.8061 ± 0.0116
k= 60 | CV AUC=0.8189 ± 0.0110
k= 80 | CV AUC=0.8221 ± 0.0097
k=146 | CV AUC=0.8257 ± 0.0101

Best K from CV: 146
Best CV AUC   : 0.8256571262095544 ± 0.010115720890134202


In [ ]:
final_mi_pipe = Pipeline([
    ("mi", SelectKBest(mutual_info_classif, k=best_k)),
    ("clf", LogisticRegression(
        max_iter=4000,
        penalty="l2",
        solver="liblinear"
    ))
])

final_mi_pipe.fit(X_train_fs, y_train)
pred_mi_test = final_mi_pipe.predict_proba(X_test_fs)[:, 1]
auc_mi_test = roc_auc_score(y_test, pred_mi_test)

print(f"Final MI + LR model with k={best_k}")
print("Test AUC:", auc_mi_test)


Final MI + LR model with k=146
Test AUC: 0.4977694895149753


In [ ]:
selector = final_mi_pipe.named_steps["mi"]
support_mask = selector.get_support()

mi_scores_all = selector.scores_
all_features = X_train_fs.columns

selected_features = all_features[support_mask]
selected_scores = mi_scores_all[support_mask]

mi_ranked = sorted(
    zip(selected_features, selected_scores),
    key=lambda x: x[1],
    reverse=True
)
top_mi_features = [name for name, score in mi_ranked[:20]]

print("\nTop MI-selected features (up to 20):")
for name, score in mi_ranked[:20]:
    print(f"{name}: {score:.4f}")



Top MI-selected features (up to 20):
LAB_PH_MIN: 0.0314
LAB_PH_MEAN: 0.0304
LAB_LACTATE_MIN: 0.0293
LAB_LACTATE_MEAN: 0.0290
LAB_LACTATE_MAX: 0.0268
AGE_x: 0.0234
ADMISSION_TYPE_EMERGENCY: 0.0207
LAB_PH_MAX: 0.0205
AGE_y: 0.0198
URINE_FLUID_RATIO: 0.0183
URINE_SUM_24H: 0.0173
LAB_CREATININE_MEAN: 0.0164
LOS: 0.0157
LAB_CREATININE_MAX: 0.0155
ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI: 0.0154
FIRST_WARDID: 0.0152
LAST_CAREUNIT_NICU: 0.0150
LAB_LACTATE_STD: 0.0147
LAB_PH_STD: 0.0144
LAB_SODIUM_MEAN: 0.0139


In [62]:
clf_l1 = LogisticRegression(
    penalty="l1",
    C=1.0,
    solver="liblinear",
    max_iter=5000
)
clf_l1.fit(X_train_fs, y_train)

coef = clf_l1.coef_.ravel()
coef_series = pd.Series(coef, index=X_train_fs.columns)

nonzero_features = coef_series[coef_series != 0].index.tolist()
print("L1-selected feature count: 20")
print("Sample L1-selected features:", nonzero_features[:30])

X_train_l1 = X_train_fs[nonzero_features[:90]]
X_test_l1  = X_test_fs[nonzero_features[:90]]

clf_l1_small = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="liblinear",
    max_iter=4000
)
clf_l1_small.fit(X_train_l1, y_train)

pred_l1 = clf_l1_small.predict_proba(X_test_l1)[:, 1]
auc_l1 = roc_auc_score(y_test, pred_l1)
print("AUC with L1-selected features:", auc_l1)


L1-selected feature count: 20
Sample L1-selected features: ['ROW_ID', 'FIRST_WARDID', 'LAST_WARDID', 'LOS', 'AGE_x', 'AGE_y', 'VITAL_HEART_RATE_MAX', 'VITAL_RESPIRATORY_RATE_MAX', 'VITAL_SPO2_MAX', 'VITAL_HEART_RATE_MEAN', 'VITAL_RESPIRATORY_RATE_MEAN', 'VITAL_SPO2_MEAN', 'VITAL_HEART_RATE_MIN', 'VITAL_RESPIRATORY_RATE_MIN', 'VITAL_SPO2_MIN', 'VITAL_HEART_RATE_STD', 'VITAL_RESPIRATORY_RATE_STD', 'VITAL_SPO2_STD', 'LAB_GLUCOSE_MAX', 'LAB_HEMOGLOBIN_MAX', 'LAB_LACTATE_MAX', 'LAB_PH_MAX', 'LAB_SODIUM_MAX', 'LAB_HEMATOCRIT_MAX', 'LAB_PTT_MAX', 'LAB_WBC_MAX', 'LAB_GLUCOSE_MEAN', 'LAB_HEMOGLOBIN_MEAN', 'LAB_LACTATE_MEAN', 'LAB_PH_MEAN']
AUC with L1-selected features: 0.49319841331879166


In [63]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)
rf.fit(X_train_fs, y_train)

rf_importances = pd.Series(rf.feature_importances_, index=X_train_fs.columns).sort_values(ascending=False)

print("Top 20 RF features:")
print(rf_importances.head(20))

K_rf = 20
top_rf_features = rf_importances.head(K_rf).index.tolist()
print("Using top-K RF features:", K_rf)

X_train_rf = X_train_fs[top_rf_features]
X_test_rf  = X_test_fs[top_rf_features]

clf_rf_fs = LogisticRegression(
    max_iter=4000,
    penalty="l2",
    solver="liblinear"
)
clf_rf_fs.fit(X_train_rf, y_train)

pred_rf_fs = clf_rf_fs.predict_proba(X_test_rf)[:, 1]
auc_rf_fs = roc_auc_score(y_test, pred_rf_fs)
print("AUC with top-%d RF features:" % K_rf, auc_rf_fs)


Top 20 RF features:
LOS                    0.050504
URINE_SUM_24H          0.033259
LAB_LACTATE_MEAN       0.031694
URINE_MEAN_24H         0.031123
LAB_PH_MEAN            0.031062
URINE_FLUID_RATIO      0.030011
LAB_PH_MIN             0.029772
LAB_LACTATE_MIN        0.028029
FLUID_MEAN_24H         0.026907
LAB_LACTATE_MAX        0.025457
FLUID_SUM_24H          0.024705
AGE_x                  0.022966
AGE_y                  0.022721
LAB_CREATININE_MEAN    0.022213
LAB_PH_MAX             0.021725
LAB_PTT_MIN            0.021463
LAB_SODIUM_MEAN        0.021423
LAB_HEMATOCRIT_MEAN    0.021154
LAB_HEMATOCRIT_MAX     0.021007
LAB_HEMATOCRIT_MIN     0.020743
dtype: float64
Using top-K RF features: 20
AUC with top-20 RF features: 0.49249118131319336


In [64]:
final_features = list(
    set(top_mi_features) |
    set(nonzero_features) |
    set(top_rf_features)
)

print("Baseline AUC (all features):", base_auc)
print("AUC with MI-selected features:", auc_mi_test)
print("AUC with L1-selected features:", auc_l1)
print("AUC with RF-selected features:", auc_rf_fs)


Baseline AUC (all features): 0.4977689518532167
AUC with MI-selected features: 0.4977694895149753
AUC with L1-selected features: 0.49319841331879166
AUC with RF-selected features: 0.49249118131319336


In [ ]:
scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()
print(f"Class imbalance ratio: {scale_pos_weight:.2f}")

from sklearn.model_selection import train_test_split
X_train_tune, X_val_tune, y_train_tune, y_val_tune = train_test_split(
    X_train_fs, y_train, test_size=0.2, random_state=42, stratify=y_train
)

if XGBClassifier is not None:
    print("\nXGBoost Hyperparameter Tuning")
    xgb_base = XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='auc',
        scale_pos_weight=scale_pos_weight,
        n_jobs=-1
    )
    
    param_dist_xgb = {
        'n_estimators': randint(300, 800),
        'max_depth': randint(4, 9),
        'learning_rate': uniform(0.01, 0.15),
        'subsample': uniform(0.6, 0.35),
        'colsample_bytree': uniform(0.6, 0.35),
        'min_child_weight': randint(1, 6),
        'gamma': uniform(0, 0.3)
    }
    
    random_search_xgb = RandomizedSearchCV(
        xgb_base, param_dist_xgb, n_iter=30, cv=3,
        scoring='roc_auc', n_jobs=-1, random_state=42, verbose=1
    )
    
    random_search_xgb.fit(X_train_tune, y_train_tune)
    
    print("Best XGBoost params:", random_search_xgb.best_params_)
    print("Best XGBoost CV AUC:", random_search_xgb.best_score_)
    
    pred_xgb_tuned = random_search_xgb.predict_proba(X_test_fs)[:, 1]
    auc_xgb_tuned = roc_auc_score(y_test, pred_xgb_tuned)
    print(f"XGBoost Test AUC (tuned): {auc_xgb_tuned:.6f}")

if LGBMClassifier is not None:
    print("\nLightGBM Hyperparameter Tuning")
    lgbm_base = LGBMClassifier(
        random_state=42,
        verbose=-1,
        scale_pos_weight=scale_pos_weight,
        n_jobs=-1
    )
    
    param_dist_lgbm = {
        'n_estimators': randint(300, 800),
        'max_depth': randint(4, 9),
        'learning_rate': uniform(0.01, 0.15),
        'num_leaves': randint(20, 50),
        'feature_fraction': uniform(0.6, 0.35),
        'bagging_fraction': uniform(0.6, 0.35),
        'bagging_freq': randint(3, 8),
        'min_child_samples': randint(10, 30),
        'min_child_weight': uniform(0.001, 0.1)
    }
    
    random_search_lgbm = RandomizedSearchCV(
        lgbm_base, param_dist_lgbm, n_iter=30, cv=3,
        scoring='roc_auc', n_jobs=-1, random_state=42, verbose=1
    )
    
    random_search_lgbm.fit(X_train_tune, y_train_tune)
    
    print("Best LightGBM params:", random_search_lgbm.best_params_)
    print("Best LightGBM CV AUC:", random_search_lgbm.best_score_)

    pred_lgbm_tuned = random_search_lgbm.predict_proba(X_test_fs)[:, 1]
    auc_lgbm_tuned = roc_auc_score(y_test, pred_lgbm_tuned)
    print(f"LightGBM Test AUC (tuned): {auc_lgbm_tuned:.6f}")

if XGBClassifier is not None and 'final_features' in locals():
    print("\nBest Model on Combined Features")
    X_train_combined = X_train_fs[final_features]
    X_test_combined = X_test_fs[final_features]
    
    best_xgb = random_search_xgb.best_estimator_
    best_xgb.fit(X_train_combined, y_train)
    pred_combined = best_xgb.predict_proba(X_test_combined)[:, 1]
    auc_combined = roc_auc_score(y_test, pred_combined)
    print(f"XGBoost on Combined Features AUC: {auc_combined:.6f}")


Class imbalance ratio: 8.94

=== XGBoost Hyperparameter Tuning ===
Fitting 3 folds for each of 30 candidates, totalling 90 fits


c:\Users\SAHIL\anaconda3\envs\dspy_env\lib\site-packages\xgboost\training.py:199: UserWarning: [16:24:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best XGBoost params: {'colsample_bytree': np.float64(0.8833253661489003), 'gamma': np.float64(0.10459979618751881), 'learning_rate': np.float64(0.02442648266371311), 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 723, 'subsample': np.float64(0.7353573712051881)}
Best XGBoost CV AUC: 0.8844205780907611
XGBoost Test AUC (tuned): 0.807505

=== LightGBM Hyperparameter Tuning ===
Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best LightGBM params: {'bagging_fraction': np.float64(0.7131210262072643), 'bagging_freq': 3, 'feature_fraction': np.float64(0.7413426598703142), 'learning_rate': np.float64(0.019733837066347234), 'max_depth': 5, 'min_child_samples': 29, 'min_child_weight': np.float64(0.02568760628386012), 'n_estimators': 594, 'num_leaves': 21}
Best LightGBM CV AUC: 0.8837980854916557
LightGBM Test AUC (tuned): 0.747945

=== Best Model on Combined Features ===


c:\Users\SAHIL\anaconda3\envs\dspy_env\lib\site-packages\xgboost\training.py:199: UserWarning: [16:29:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost on Combined Features AUC: 0.791912
